# 16 Puzzle Solver
* The 15-puzzle is a sliding puzzle that consists of a frame of
numbered square tiles in random order with one tile missing.
* Example of 8-puzzle is in https://github.com/aimacode/aima-python/blob/master/search.py.
* Our goal is to solve it using A*(A-star) algorithm, and use h1, h2 as herustic functions to compare their efficiency.

## Import the relative utils
* random 用於生成隨機的 16-Puzzle
* typing 用於確保變數以及函數參數跟回傳值
* heapq 用在 Best-First-Search 中，作為 Priority Queue

In [80]:
import sys
import time
import random
from typing import *
from heapq import *
import heapq

## Creating the 16-Puzzle Problem as a Class
* 為了方便，我們建立一個 Class 內含 `is_solvable()`, `generate_random_state()`, 以及 `to_matrix_list()` 等方法:
  - `is_solvable()`: 輸入（可不輸入）一個 `state`， 如果 `state` 是 `None`，將會直接採用當前 class 內的 state `self.state`。 輸出給定 puzzle 是否可解，如果可以回傳 `True`，反之 `False`
    - 反序數（Inversions）：反序數是指在排列中，出現一對數字 (a) 和 (b)，其中 (a > b)，但 (a) 出現在 (b) 的前面。
    - 空格的位置（Blank Position）：空格（即數字 0）所在的行數（從底部開始計算）。
    - 判斷規則:
      * 對於 (N \times N) 的拼圖（例如 16-puzzle 是 (4 \times 4)）：

        - 如果 (N) 是奇數，則僅需檢查反序數是否為偶數。
        - 如果 (N) 是偶數，則需要檢查以下條件：
          - 反序數的奇偶性。
          - 空格所在行數（從底部開始計算）的奇偶性。
      * 具體規則如下：

        - 如果反序數是偶數，且空格所在行數是奇數，則拼圖可解。
        - 如果反序數是奇數，且空格所在行數是偶數，則拼圖可解。
    
  - `generate_random_state()`: 不用輸入，輸出一個 state ，同時會更新當前 Class 內的 state `self.state` ，生成過程中，每次生成都會檢查生成出來的結果是否可解，若不行則會繼續再生一次，直到可以解為止，對於一個 16-Puzzle 問題，大概有 16! 種組合，其中大概一半可以解，所以此方法所花時間照理來說不會太多
  
  - `to_matrix_list()`: 輸入（可不輸入）一個 `state`， 如果 `state` 是 `None`，將會直接採用當前 class 內的 state `self.state`。 輸出一個變成 matrix 形式的 Puzzle。
    - 由於此次問題參照 Github Repo: https://github.com/aimacode/aima-python/blob/master/search.py 裡面的範例，所以採用壓平後的 Tuple 作為資料結構而不是直接用二維矩陣，因此在運算上也會有點不一樣：
      - 移動：
        * 往左為當前位置 x 座標 -1，
        * 往右為當前位置 x 座標 +1，
        * 往上為當前位置 y 座標 -4，
        * 往下為當前位置 y 座標 +4，
      - 邊界檢查：如果當前在 Tuple 中的 index
        * $index % 4 == 0$：代表位於左界
        * $index < 4$：代表位於上界
        * $index % 4 == 3$：代表位於右界
        * $index > 11$：代表位於下界

In [81]:
class SixteenPuzzleProblem():
    def __init__(
      self,
      init_state: List[int] | Tuple[int] = None
    ):
      if (type(init_state) == list): init_state = tuple(init_state)

      self.goal_state = (
        1, 2, 3, 4,
        5, 6, 7, 8,
        9, 10, 11, 12,
        13, 14, 15, 0
      )
      self.state = (
        1, 2, 3, 4,
        5, 6, 7, 8,
        9, 10, 11, 12,
        13, 14, 15, 0
      ) if not init_state or not self.is_solvable(init_state) \
        else init_state
      print(self.state)

    def is_solvable(
      self,
      state: Tuple[int] = None
    ) -> bool:
      state = self.state if not state else state

      inversion = 0
      for i in range(len(state)):
        for j in range(i + 1, len(state)):
          if (state[i] > state[j]) and state[i] != 0 and state[j] != 0:
            inversion += 1

      blank_row = 4 - (state.index(0) // 4)

      return (inversion % 2 == 0) == (blank_row % 2 == 1)

    def generate_random_state(self) -> Tuple[int]:
      state = tuple(random.sample(range(16), 16))
      while not self.is_solvable():
        print("Unsolvable puzzle detected, re-generating new puzzle...")
        state = tuple(random.sample(range(16), 16))

      print("Solvable puzzle generated: ", state)
      self.state = state
      return self.state

    def to_matrix_list(self, state: Tuple[int] = None) -> List[List[int]]:
      if not state: state = self.state

      matrix = []
      size = 4
      for i in range(0, 16, size):
          matrix.append(list(self.state[i:i+size]))
      return matrix

## Creating the 16-Puzzle Solver as a Class
* 透過 A*(A-star) 演算法求解，其中透過 Best-First-Search 實作搜尋的關鍵函數。
* h1 為計算錯位總數的 Heuristic 函數
* h2 為計算個方塊距離目標位置的 Hamilton 距離加總之 Heuristic 函數

In [82]:
class SixteenPuzzleSolver:
    def __init__(self, goal_state: Tuple[int]):
        self.goal_state = goal_state
        self.size = 4
        self.directions = {
            'UP': -self.size,
            'DOWN': self.size,
            'LEFT': -1,
            'RIGHT': 1
        }

    def __get_available_actions__(self, state: Tuple[int]) -> List[str]:
        possible_actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        index = state.index(0)

        if index % 4 == 0:
            possible_actions.remove('LEFT')
        if index < 4:
            possible_actions.remove('UP')
        if index % 4 == 3:
            possible_actions.remove('RIGHT')
        if index > 11:
            possible_actions.remove('DOWN')

        return possible_actions

    def __best_first_search__(self, init_state: Tuple[int], h = lambda x: 0):
        pq = []
        heapq.heappush(pq, (h(state=init_state, goal_state=self.goal_state), 0, init_state, 0))
        visited = {}

        while pq:
            f, g, state, count = heapq.heappop(pq)

            if state == self.goal_state:
              return count

            if state in visited and visited[state] <= g:
              continue

            visited[state] = g

            zero_index = state.index(0)

            for direction in self.__get_available_actions__(list(state)):
                delta = self.directions[direction]
                new_zero_index = zero_index + delta

                new_state_list = list(state)
                new_state_list[zero_index], new_state_list[new_zero_index] = (
                    new_state_list[new_zero_index], new_state_list[zero_index]
                )
                new_state = tuple(new_state_list)

                new_g = g + 1
                new_f = new_g + h(state=new_state, goal_state=self.goal_state)

                heapq.heappush(pq, (new_f, new_g, new_state, count + 1))

        return None

    def astar_search(self, init_state: Tuple[int], h = lambda x: 0):
        return self.__best_first_search__(init_state, h)

In [83]:
def h1(state: Tuple[int], goal_state: Tuple[int]):
  return sum(i != j for i in state for j in goal_state)

In [84]:
def h2(state: Tuple[int], goal_state: Tuple[int]):
  return sum(abs(i // 4 - goal_state.index(val) // 4) + abs(i % 4 - goal_state.index(val) % 4)
    for i, val in enumerate(state))

## Generate the Problem
* 使用我們剛剛已經建立好的 `SixteenPuzzleProblem` Class 來建立一個 `problem` 實例。
* 注意：有很多 Problem 可能會花費非常多的時間解決，建議採用我在下方註解掉的測資來測試並比較 h1 跟 h2 的效率差別。

In [122]:
problem = SixteenPuzzleProblem(
    (2, 8, 3, 4, 1, 6, 0, 7, 5, 9, 10, 11, 13, 14, 15, 12)
)
# problem.generate_random_state()
print(problem.to_matrix_list())

# Is puzzle solvable?
print("Is puzzle solvable?", problem.is_solvable(problem.state))

(2, 8, 3, 4, 1, 6, 0, 7, 5, 9, 10, 11, 13, 14, 15, 12)
[[2, 8, 3, 4], [1, 6, 0, 7], [5, 9, 10, 11], [13, 14, 15, 12]]
Is puzzle solvable? True


## Initialize the SixteenPuzzleSolver Class

In [86]:
solver = SixteenPuzzleSolver(goal_state=problem.goal_state)

## Solving Using h1 as Heuristic Function, and Calculate the Efficiency

In [123]:
start_time_1 = time.time()

step_count_1 = solver.astar_search(problem.state, h1)

end_time_1 = time.time()
print(f"h1 (錯位數) 共執行: {step_count_1} 個步驟, 耗時: {end_time_1 - start_time_1:.4f} 秒")

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0) (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
h1 (錯位數) 共執行: 17 個步驟, 耗時: 24.5776 秒


## Solving Using h2 as Heuristic Function, and Calculate the Efficiency

In [126]:
start_time_2 = time.time()

step_count_2 = solver.astar_search(problem.state, h2)

end_time_2 = time.time()
print(f"h2 (曼哈頓距離) 共執行: {step_count_2} 個步驟, 耗時: {end_time_2 - start_time_2:.4f} 秒")

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0) (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
h2 (曼哈頓距離) 共執行: 17 個步驟, 耗時: 0.0221 秒
